Install libraries

In [299]:
!pip install sentinelsat
!pip install geopandas
!pip install rasterio

     |████████████████████████████████| 18.1MB 215kB/s 


Import libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from sentinelsat import SentinelAPI
import folium
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
import urllib.request as request
import json
from datetime import datetime
import zipfile
import rasterio as rio

Extract Geometries

In [309]:
max_records=10
first_record = 18290
current_record = first_record
names = []
countries = []
latitudes = []
longitudes = []
geometries = []
while len(names) < max_records:
  water_url = f'https://water.blue-dot-observatory.com/api/waterbodies/{current_record}/index.html'
  try:
    water_body_data = json.loads(request.urlopen(water_url).read().decode())
    names.append(water_body_data['properties']['name'])
    countries.append(water_body_data['properties']['country'])
    latitudes.append(water_body_data['properties']['lat'])
    longitudes.append(water_body_data['properties']['long'])
    geometries.append(water_body_data['nominal_outline']['geometry'])
  except:
    pass
  current_record += 1

https://water.blue-dot-observatory.com/api/waterbodies/18290/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18291/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18292/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18293/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18294/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18295/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18296/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18297/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18298/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18299/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18300/index.html
https://water.blue-dot-observatory.com/api/waterbodies/18301/index.html


Create Dataframe

In [0]:
water_bodies = pd.DataFrame({'Name':names,
                             'Country':countries,
                             'Latitude':latitudes,
                             'Longitude':longitudes,
                             'Geometry':geometries})

In [312]:
water_bodies

,Name,Country,Latitude,Longitude,Geometry
0,Lake Travis,United States of America,30.442056,-98.004388,"{'coordinates': [[[[-98.0217194, 30.3745221], ..."
1,Lake Arlington,United States of America,32.703161,-97.213846,"{'coordinates': [[[-97.2335181, 32.6921516], [..."
2,Lake Buchanan,United States of America,30.818283,-98.422734,"{'coordinates': [[[-98.4783375, 30.7971736], [..."
3,Walter E Long Lake,United States of America,30.297869,-97.601663,"{'coordinates': [[[-97.6199675, 30.303003], [-..."
4,Cooling Water Reservoir,United States of America,28.765775,-96.049176,"{'coordinates': [[[-96.080274, 28.7517011], [-..."
5,Lake Tawakoni,United States of America,32.882111,-95.991711,"{'coordinates': [[[-96.1319487, 32.8534733], [..."
6,Eagle Mountain Dam,United States of America,32.919427,-97.497142,"{'coordinates': [[[-97.529902, 32.9085778], [-..."
7,Twin Buttes,United States of America,31.358521,-100.543715,"{'coordinates': [[[[-100.6083856, 31.3296567],..."
8,Peacock Site 1-A Tailings Dam,United States of America,32.969099,-94.674863,"{'coordinates': [[[-94.6819945, 32.9681979], [..."
9,Somerville Dam,United States of America,30.311131,-96.586281,"{'coordinates': [[[[-96.5679466, 30.2964676], ..."


In [0]:
lake_travis = shape(water_bodies.iloc[0]['Geometry'])

Plot Map

In [0]:
my_map = folium.Map(location = [30.442056, -98.004388], zoom_start=11)

In [158]:
folium.GeoJson(lake_travis).add_to(my_map)
my_map

Read login credentials

In [0]:
with open('/content/drive/My Drive/Sentinel Hub Credentials/login_info.txt') as file:
  credentials = file.readlines()
  credentials = [credential.strip() for credential in credentials]

Download Satellite Images

In [0]:
for water_body_geometry in water_bodies['Geometry']:
  water_body_shape = shape(water_body_geometry)
  shape_simplified = water_body_shape.simplify(1, preserve_topology=True)

In [0]:
api = SentinelAPI(credentials[0], credentials[1], 'https://scihub.copernicus.eu/dhus')

In [0]:
today = datetime.today().strftime('%Y%m%d')

In [0]:
results = api.query(polygon_simplified, date = ('20140101', today), platformname = 'Sentinel-2', processinglevel = 'Level-2A', cloudcoverpercentage = (0,5))

In [291]:
results_gdf = api.to_geodataframe(results).sort_values(['cloudcoverpercentage'], ascending=True)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
uuid = results_gdf.iloc[0]['uuid']

In [0]:
filename = '/content/' + results_gdf.iloc[0]['title'] + '.zip'

In [296]:
api.download(uuid)

{'Creation Date': datetime.datetime(2019, 12, 2, 0, 32, 12, 905000),
 'Ingestion Date': datetime.datetime(2019, 12, 2, 0, 31, 25, 67000),
 'Online': True,
 'date': datetime.datetime(2019, 12, 1, 17, 16, 49, 24000),
 'downloaded_bytes': 0,
 'footprint': 'POLYGON((-99.00021 30.73307003214313,-97.85333 30.728004669372204,-97.86478 29.73733482256821,-99.00021 29.74220376506903,-99.00021 30.73307003214313))',
 'id': '598a8c78-1fb6-4948-9ebb-b47e2ebdef4e',
 'md5': '38C9D1C43991317B39290D9C69B83082',
 'path': './S2B_MSIL2A_20191201T171649_N0213_R112_T14RNU_20191201T195554.zip',
 'size': 1206221194,
 'title': 'S2B_MSIL2A_20191201T171649_N0213_R112_T14RNU_20191201T195554',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('598a8c78-1fb6-4948-9ebb-b47e2ebdef4e')/$value"}

In [0]:
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/Satellite Data/')